In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import keras

In [6]:
X_train = pd.read_csv('X_train.csv').values
Y_train = pd.read_csv('Y_train.csv').values
print(X_train)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)

X_train_split, X_test_split, Y_train_split, Y_test_split = train_test_split(
    X_train_tensor, Y_train_tensor, test_size=0.15, random_state=42
)

train_dataset = TensorDataset(X_train_split, Y_train_split)
test_dataset = TensorDataset(X_test_split, Y_test_split)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

[['...']]


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
# model = keras.Sequential()
# model.add(keras.Input(shape=(300,)))
# model.add(keras.layers.Dense())
# model.compile(optimizer='sgd', loss='mse')
# model.fit(X_train, Y_train, batch_size = 32, epochs = 10)


TypeError: Sequential.__init__() got an unexpected keyword argument 'trainable'

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(50 * 6, 250)
        self.fc2 = nn.Linear(250, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 6)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return torch.softmax(x, dim=1)

In [ ]:
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum = .9)
def train(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            # Convert one-hot labels to class indices
            labels_indices = labels.argmax(dim=1)
            loss = criterion(outputs, labels_indices)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

# Evaluation function
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            # Convert one-hot labels to class indices
            labels_indices = labels.argmax(dim=1)
            test_loss += criterion(outputs, labels_indices).item()
            preds = outputs.argmax(dim=1)
            correct += (preds == labels_indices).sum().item()
    accuracy = correct / len(test_loader.dataset)
    print(f"Test Loss: {test_loss / len(test_loader)}, Accuracy: {accuracy}")


In [ ]:
train(model, train_loader, criterion, optimizer, epochs=100)

evaluate(model, test_loader, criterion)

Epoch 1, Loss: 1.790362592096682
Epoch 2, Loss: 1.7900380911650482
Epoch 3, Loss: 1.789620867481938
Epoch 4, Loss: 1.7892797125710382
Epoch 5, Loss: 1.7891416240621496
Epoch 6, Loss: 1.7888634734683566
Epoch 7, Loss: 1.7883821460935805
Epoch 8, Loss: 1.7880272644537467
Epoch 9, Loss: 1.7877782362478751
Epoch 10, Loss: 1.7876070737838745
Epoch 11, Loss: 1.7873070858142994
Epoch 12, Loss: 1.7870352886341236
Epoch 13, Loss: 1.7868251314869634
Epoch 14, Loss: 1.7864476574791803
Epoch 15, Loss: 1.7864533397886488
Epoch 16, Loss: 1.7861032441810325
Epoch 17, Loss: 1.785972246417293
Epoch 18, Loss: 1.7858302769837555
Epoch 19, Loss: 1.7855682019834165
Epoch 20, Loss: 1.7851557157657765
Epoch 21, Loss: 1.784788348056652
Epoch 22, Loss: 1.7847390748836376
Epoch 23, Loss: 1.78428135977851
Epoch 24, Loss: 1.7842554339656123
Epoch 25, Loss: 1.7842203952648021
Epoch 26, Loss: 1.784237888124254
Epoch 27, Loss: 1.783591460298609
Epoch 28, Loss: 1.7834210484116166
Epoch 29, Loss: 1.7828064097298517
Ep